#### Data Processing

In [115]:
# initialization
import pandas as pd
import numpy as np
import itertools

data = pd.read_csv("initialdata_v3.csv")
# data = pd.read_csv("initialdata.csv")

In [116]:
data

,Age,Gender,Married,Default
0,24,M,Y,Y
1,30,F,Y,Y
2,31,F,Y,N
3,42,M,Y,N
4,17,M,N,Y
5,16,F,N,Y
6,18,M,N,N
7,19,M,N,N
8,29,M,Y,Y
9,35,F,Y,N


In [117]:
# encoding using one-hot encoder

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()


## Binary calculation
# Default
enc.fit(np.array(data['Default']).reshape(-1, 1))
ohe = enc.transform(np.array(data['Default']).reshape(-1, 1)).toarray()
# enc.categories_
ohe_fixed = []
for i in ohe:
#     print(i[1])
    ohe_fixed.append(i[1])

# Gender
enc.fit(np.array(data['Gender']).reshape(-1, 1))
ohe = enc.transform(np.array(data['Gender']).reshape(-1, 1)).toarray()
# enc.categories_
ohe_fixed2 = []
for i in ohe:
#     print(i[1])
    ohe_fixed2.append(i[1])
    
# Married
enc.fit(np.array(data['Married']).reshape(-1, 1))
ohe = enc.transform(np.array(data['Married']).reshape(-1, 1)).toarray()
# enc.categories_
ohe_fixed4 = []
for i in ohe:
#     print(i[1])
    ohe_fixed4.append(i[1])
        
        
    
## Mean based calculation
# Age
mean = data['Age'].mean()
ohe_fixed3 = []
for index, row in data.iterrows():
    if row['Age'] >= mean:
        ohe_fixed3.append(1)
    if row['Age'] < mean:
        ohe_fixed3.append(0)

encodedDataComplete = pd.DataFrame()
encodedDataComplete['Age'] = ohe_fixed3
encodedDataComplete['Gender'] = ohe_fixed2
encodedDataComplete['Married'] = ohe_fixed4
encodedDataComplete['Default'] = ohe_fixed

## not removing duplicates will increase accuracy rate
# encodedDataComplete = encodedDataComplete.drop_duplicates()

In [118]:
encodedDataComplete

,Age,Gender,Married,Default
0,0,1.0,1.0,1.0
1,1,0.0,1.0,1.0
2,1,0.0,1.0,0.0
3,1,1.0,1.0,0.0
4,0,1.0,0.0,1.0
5,0,0.0,0.0,1.0
6,0,1.0,0.0,0.0
7,0,1.0,0.0,0.0
8,1,1.0,1.0,1.0
9,1,0.0,1.0,0.0


In [90]:
# encoding using dummies

encodedData = pd.DataFrame()
for col in data.columns:
    df = pd.get_dummies(data[col]) # one hot encoding
    for newcol in df.columns:
        name = str(col)+"_"+str(newcol)
        encodedData[name] = df[newcol]
        
# single output column
encodedDataComplete = encodedData.drop("Default_N", axis=1)

In [5]:
# encodedDataComplete = encodedDataComplete[0:] # filtering; debugging only!

In [6]:
# # testing dataframe - should be 100% accuracy
# encodedDataComplete = pd.DataFrame([[0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0]], columns=['hi', 'hi2', 'hi3', 'hi4'])
# encodedDataComplete

In [119]:
xCol = encodedDataComplete.columns[0:-1]
yCol = encodedDataComplete.columns[-1]
outputData = encodedDataComplete[yCol]
encodedData = encodedDataComplete[xCol]

#### Neural Logic Network

In [120]:
# logic gate learning

import numpy
import scipy.special
import glob
import scipy.misc
class neuralNetwork:
    def __init__(self, inputNodes, hiddenOneNodes, hiddenTwoNodes, hiddenThreeNodes, finalNodes, alpha):
        self.inputNodes = inputNodes
        self.hiddenOneNodes = hiddenOneNodes
        self.hiddenTwoNodes = hiddenTwoNodes
        self.hiddenThreeNodes = hiddenThreeNodes
        self.finalNodes = finalNodes
        self.alpha = alpha
        self.weightsInputHidden = numpy.random.normal(0.0, pow(self.hiddenOneNodes, -0.5),(self.hiddenOneNodes,self.inputNodes))
        self.weightsHiddenOneHiddenTwo = numpy.random.normal(0.0, pow(self.hiddenTwoNodes,-0.5),(self.hiddenTwoNodes,self.hiddenOneNodes))
        self.weightsHiddenTwoHiddenThree = numpy.random.normal(0.0, pow(self.hiddenThreeNodes,-0.5),(self.hiddenThreeNodes,self.hiddenTwoNodes))
        self.weightsHiddenOutput = numpy.random.normal(0.0, pow(self.hiddenOneNodes,-0.5),(self.finalNodes, self.hiddenThreeNodes))
        pass
    def train(self, inputs, target):
        inputs = numpy.array(inputs, ndmin=2).T
        target = numpy.array(target, ndmin=2).T
        hiddenInput = numpy.dot(self.weightsInputHidden,inputs)
        hiddenOneOutput = self.sigmoid(hiddenInput)
        hiddenTwoInput = numpy.dot(self.weightsHiddenOneHiddenTwo,hiddenOneOutput)
        hiddenTwoOutput = self.sigmoid(hiddenTwoInput)
        hiddenThreeInput = numpy.dot(self.weightsHiddenTwoHiddenThree,hiddenTwoOutput)
        hiddenThreeOutput = self.sigmoid(hiddenThreeInput)
        finalInput = numpy.dot(self.weightsHiddenOutput,hiddenThreeOutput)
        finalOutput = self.sigmoid(finalInput)
        outputError = target - finalOutput
        hiddenOutputError = numpy.dot(self.weightsHiddenOutput.T, outputError)
        hiddenThreeHiddenTwoError = numpy.dot(self.weightsHiddenTwoHiddenThree.T, hiddenOutputError)
        hiddenTwoHiddenOneError = numpy.dot(self.weightsHiddenOneHiddenTwo.T, hiddenThreeHiddenTwoError)
        hiddenInputError = numpy.dot(self.weightsInputHidden.T, hiddenTwoHiddenOneError)
        self.weightsHiddenOutput += self.alpha * numpy.dot((outputError * finalOutput * (1.0 - finalOutput)),numpy.transpose(hiddenThreeOutput))
        self.weightsHiddenTwoHiddenThree += self.alpha * numpy.dot((hiddenOutputError * hiddenThreeOutput * (1.0 - hiddenThreeOutput)),numpy.transpose(hiddenTwoOutput))
        self.weightsHiddenOneHiddenTwo += self.alpha * numpy.dot((hiddenThreeHiddenTwoError * hiddenTwoOutput * (1.0 - hiddenTwoOutput)),numpy.transpose(hiddenOneOutput))
        self.weightsInputHidden += self.alpha * numpy.dot((hiddenTwoHiddenOneError * hiddenOneOutput * (1.0 - hiddenOneOutput)),numpy.transpose(inputs))        
        pass
    def query(self, inputs):
        inputs = numpy.array(inputs, ndmin=2).T
        hiddenInput = numpy.dot(self.weightsInputHidden,inputs)
        hiddenOneOutput = self.sigmoid(hiddenInput)
        hiddenTwoInput = numpy.dot(self.weightsHiddenOneHiddenTwo,hiddenOneOutput)
        hiddenTwoOutput = self.sigmoid(hiddenTwoInput)
        hiddenThreeInput = numpy.dot(self.weightsHiddenTwoHiddenThree,hiddenTwoOutput)
        hiddenThreeOutput = self.sigmoid(hiddenThreeInput)
        finalInput = numpy.dot(self.weightsHiddenOutput,hiddenThreeOutput)
        finalOutput = self.sigmoid(finalInput)
        return finalOutput
        pass
    def sigmoid(self, x):
        return scipy.special.expit(x)
        pass

In [121]:
#AND
nnAND = neuralNetwork(2,12,36,12,1,0.1)
print('Before training')
print(nnAND.query([0,0]))
print(nnAND.query([0,1]))
print(nnAND.query([1,0]))
print(nnAND.query([1,1]))
print("Training...")
for i in range(0, 10000):
    nnAND.train([0,0],[0])
    nnAND.train([0,1],[0])
    nnAND.train([1,0],[0])
    nnAND.train([1,1],[1])
print("Done")
print(nnAND.query([0,0]))
print(nnAND.query([0,1]))
print(nnAND.query([1,0]))
print(nnAND.query([1,1]))

Before training
[[0.46540099]]
[[0.46550479]]
[[0.46485937]]
[[0.46496519]]
Training...
Done
[[0.00503326]]
[[0.00503618]]
[[0.00503159]]
[[0.9899706]]


In [122]:
#OR
nnOR = neuralNetwork(2,12,36,12,1,0.1)
print('Before training')
print(nnOR.query([0,0]))
print(nnOR.query([0,1]))
print(nnOR.query([1,0]))
print(nnOR.query([1,1]))
print("Training...")
for i in range(0, 10000):
    nnOR.train([0,0],[0])
    nnOR.train([0,1],[1])
    nnOR.train([1,0],[1])
    nnOR.train([1,1],[1])
print("Done")
print(nnOR.query([0,0]))
print(nnOR.query([0,1]))
print(nnOR.query([1,0]))
print(nnOR.query([1,1]))

Before training
[[0.43541947]]
[[0.43532418]]
[[0.43492662]]
[[0.43486109]]
Training...
Done
[[0.01021573]]
[[0.99511345]]
[[0.99511345]]
[[0.99511373]]


In [123]:
#NOT
nnNOT = neuralNetwork(1,12,36,12,1,0.1)
print('Before training')
print(nnNOT.query([0]))
print(nnNOT.query([1]))
print("Training...")
for i in range(0, 10000):
    nnNOT.train([0],[1])
    nnNOT.train([1],[0])
print("Done")
print(nnNOT.query([0]))
print(nnNOT.query([1]))

Before training
[[0.39455162]]
[[0.39532105]]
Training...
Done
[[0.99099439]]
[[0.00894904]]


In [124]:
#NAND
nnNAND = neuralNetwork(2,12,36,12,1,0.1)
print('Before training')
print(nnNAND.query([0,0]))
print(nnNAND.query([0,1]))
print(nnNAND.query([1,0]))
print(nnNAND.query([1,1]))
print("Training...")
for i in range(0, 10000):
    nnNAND.train([0,0],[1])
    nnNAND.train([0,1],[1])
    nnNAND.train([1,0],[1])
    nnNAND.train([1,1],[0])
print("Done")
print(nnNAND.query([0,0]))
print(nnNAND.query([0,1]))
print(nnNAND.query([1,0]))
print(nnNAND.query([1,1]))

Before training
[[0.62147957]]
[[0.62139226]]
[[0.62176205]]
[[0.6216696]]
Training...
Done
[[0.99536501]]
[[0.99537011]]
[[0.99535685]]
[[0.01108328]]


In [125]:
#XOR
nnXOR = neuralNetwork(2,12,36,12,1,0.1)
print('Before training')
print(nnXOR.query([0,0]))
print(nnXOR.query([0,1]))
print(nnXOR.query([1,0]))
print(nnXOR.query([1,1]))
print("Training...")
for i in range(0, 10000):
    nnXOR.train([0,0],[0])
    nnXOR.train([0,1],[1])
    nnXOR.train([1,0],[1])
    nnXOR.train([1,1],[0])
print("Done")
print(nnXOR.query([0,0]))
print(nnXOR.query([0,1]))
print(nnXOR.query([1,0]))
print(nnXOR.query([1,1]))

Before training
[[0.35920967]]
[[0.35850979]]
[[0.36065019]]
[[0.35992839]]
Training...
Done
[[0.00611695]]
[[0.99223528]]
[[0.99207031]]
[[0.0062403]]


In [47]:
encodedDataComplete = encodedDataComplete.sample(frac=1).reset_index(drop=True)
xCol = encodedDataComplete.columns[0:-1]
yCol = encodedDataComplete.columns[-1]
outputData = encodedDataComplete[yCol]
encodedData = encodedDataComplete[xCol]
# encodedData
# outputData
varName = list(encodedData.columns)
varName

['Age', 'Gender', 'Married']

In [98]:
# NLN Training on use case

## select two variables > select logic gate > get output and repeat until result found > if result not match, continue
## if result  match, try the same logic gate sructure with all the other rows, store accuracy score (how many match)

# parse variable by variable
## Question: should we only go through single AGe var, or all of them? 
## -- probably all of them, or else how distinguish one 1 from another?
## -- does not being of a certain age add information bits?
## -- age is already given and can be used to auto-filter; why unfilter?

from random import randint
import itertools
from tqdm import tnrange, tqdm_notebook, tqdm
from time import sleep
# for index, row in encodedData.iterrows():
# for i in tnrange(3, desc='1st loop'):
#     for j in tqdm_notebook(range(100), desc='2nd loop'):
#         sleep(0.01)

logicName = ['AND', 'OR', 'NAND', 'XOR']
logicNN = [nnAND, nnOR, nnNAND, nnXOR] # nnNOT does not work at the moment
# logicName = ['AND', 'OR', 'XOR']
# logicNN = [nnAND, nnOR, nnXOR] # nnNOT does not work at the moment

logicHistory = []
varHistory = []
logicnameHistory = []

varLogs = []
logicLogs = []
logicnameLogs = []
accuracyLogs = []

varName = list(encodedData.columns)
placeholderList = []

# select variables
for i in tnrange(1000): # n iterations

    # shuffle order in each iteration + reindex
#     encodedDataComplete = encodedDataComplete.sample(frac=1).reset_index(drop=True)
#     xCol = encodedDataComplete.columns[0:-1]
#     yCol = encodedDataComplete.columns[-1]
#     outputData = encodedDataComplete[yCol]
#     encodedData = encodedDataComplete[xCol]
    
#     varName = list(encodedData.columns)
    
# for i in tnrange(len(varName)):
    for var in varName:

#         var = varName[i]
        placeholderList = [item for item in varName if item!=var]

        varHistory.append(var)

        for var2 in placeholderList:
            placeholderList = [item for item in placeholderList if item!=var2]

            # variables selected > select logic gate
            randomIndex = randint(0, len(logicName)-1)
            gate = logicNN[randomIndex]

            input1 = encodedData[var][0]
            input2 = encodedData[var2][0]

            output = gate.query([input1, input2])

            logicHistory.append(gate); logicnameHistory.append(logicName[randomIndex])
            varHistory.append(var2)
            
#             print(varHistory)
#             print(logicName[randomIndex])

            # use output with next var in placeholderlist
            if len(varName) > 2:
                for varZ in placeholderList:
                    placeholderList = [item for item in placeholderList if item!=varZ]
                    randomIndex = randint(0, len(logicName)-1)
                    gate = logicNN[randomIndex]
                    inputZ = encodedData[varZ][0]

                    output = gate.query([output[0][0], inputZ])
        #             print(placeholderList)
        #             print(output)
#                     print(varZ)

                    logicHistory.append(gate); logicnameHistory.append(logicName[randomIndex])
                    varHistory.append(varZ)

                    # check output
                    if varZ == list(encodedData.columns)[-1]:

                        if round(output[0][0]) == outputData[0]: # output match -- now check how many rows of data does the logic model fit
        #                     print([output[0][0], outputData[0]])

                            # execute for all rows, generate accuracy score

                            for index, row in encodedDataComplete.iterrows():
                                correctCount = 0
                                intCounter = 0
                                v1 = row[varHistory[intCounter]]
                                while intCounter < len(varHistory)-1:
    #                                 try: 
    #                                 print(len(varHistory)-1)
    #                                 print(intCounter)
    #                                 print(intCounter+1)
                                    lg = logicHistory[intCounter]
                                    intCounter +=1
                                    v2 = row[varHistory[intCounter]]
        #                             print(lg)
                                    otpt = lg.query([v1, v2])[0][0]
                                    v1 = otpt
    #                                 print(intCounter)
    #                                 except:
    #                                     continue

#                                 print(True)
                                if round(otpt) == row[list(encodedDataComplete.columns)[-1]]:
                                    correctCount+=1

                            # store accuracy score, store history logs
                            accuracy = correctCount / float(len(encodedDataComplete))
                    
#                             if accuracy < 0.5:
#                                 # inverse the accuracy; i.e. just do opposite of the predicted output!
#                                 for index, row in encodedDataComplete.iterrows():
#                                     correctCount = 0
#                                     intCounter = 0
#                                     v1 = row[varHistory[intCounter]]
#                                     while intCounter < len(varHistory)-1:
#                                         lg = logicHistory[intCounter]
#                                         intCounter +=1
#                                         v2 = row[varHistory[intCounter]]
#                                         otpt = lg.query([v1, v2])[0][0]
#                                         v1 = otpt
#                                     otpt = 1.0 - otpt
#                                     if round(otpt) == row[list(encodedDataComplete.columns)[-1]]:
#                                         correctCount+=1            
                            
#                             accuracy = correctCount / float(len(encodedDataComplete))
                    
                            if accuracy < 0.5:
                                theoretical_inverse_accuracy = 1.0-accuracy
                                accuracyLogs.append(theoretical_inverse_accuracy)
                            if accuracy >= 0.5:
                                accuracyLogs.append(accuracy)
            
                            varLogs.append(varHistory)
                            logicLogs.append(logicHistory)
                            logicnameLogs.append(logicnameHistory)

    #                     else:

            
            if len(varName) <= 2:

                if var2 == list(encodedData.columns)[-1]:

                    if round(output[0][0]) == outputData[0]: # output match -- now check how many rows of data does the logic model fit
    #                     print([output[0][0], outputData[0]])

                        # execute for all rows, generate accuracy score

                        for index, row in encodedDataComplete.iterrows():
                            correctCount = 0
                            intCounter = 0
                            v1 = row[varHistory[intCounter]]
                            while intCounter < len(varHistory)-1:
    #                                 try: 
    #                                 print(len(varHistory)-1)
    #                                 print(intCounter)
    #                                 print(intCounter+1)
                                lg = logicHistory[intCounter]
                                intCounter +=1
                                v2 = row[varHistory[intCounter]]
    #                             print(lg)
                                otpt = lg.query([v1, v2])[0][0]
                                v1 = otpt
    #                                 print(intCounter)
    #                                 except:
    #                                     continue

#                             print(True)
                            if round(otpt) == row[list(encodedDataComplete.columns)[-1]]:
                                correctCount+=1
#                                 print(round(otpt))
#                                 print(row[list(encodedDataComplete.columns)[-1]])

                        # store accuracy score, store history logs
                        accuracy = correctCount / float(len(encodedDataComplete))
#                         print(accuracy)
#                         if accuracy < 0.5:
# #                             print(True)
#                             # inverse the accuracy; i.e. just do opposite of the predicted output!
#                             for index, row in encodedDataComplete.iterrows():
#                                 correctCount = 0
#                                 intCounter = 0
#                                 v1 = row[varHistory[intCounter]]
#                                 while intCounter < len(varHistory)-1:
#                                     lg = logicHistory[intCounter]
#                                     intCounter +=1
#                                     v2 = row[varHistory[intCounter]]
#                                     otpt = lg.query([v1, v2])[0][0]
#                                     v1 = otpt
# #                                 print(otpt)
# #                                 otpt = 1.0 - otpt
# #                                 print(row[list(encodedDataComplete.columns)[-1]])
# #                                 print(round(otpt))
#                                 if round(otpt) != row[list(encodedDataComplete.columns)[-1]]:
#                                     print(1.0 - otpt)
#                                     correctCount+=1            

#                         accuracy = correctCount / float(len(encodedDataComplete))
#                         print(accuracy)

                        if accuracy < 0.5:
                                theoretical_inverse_accuracy = 1.0-accuracy
                                accuracyLogs.append(theoretical_inverse_accuracy)
                        if accuracy >= 0.5:
                            accuracyLogs.append(accuracy)
                    
#                         accuracyLogs.append(accuracy)
                        varLogs.append(varHistory)
                        logicLogs.append(logicHistory)
                        logicnameLogs.append(logicnameHistory)

        logicHistory = []
        logicnameHistory = []
        varHistory = []

    print("Iteration " + str(i+1))
            
        
        ### logs
#         print(logicName[randomIndex])
#         print([var, var2])
#         print([input1, input2])
#         print(output)

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration 77
Iteratio

Iteration 593
Iteration 594
Iteration 595
Iteration 596
Iteration 597
Iteration 598
Iteration 599
Iteration 600
Iteration 601
Iteration 602
Iteration 603
Iteration 604
Iteration 605
Iteration 606
Iteration 607
Iteration 608
Iteration 609
Iteration 610
Iteration 611
Iteration 612
Iteration 613
Iteration 614
Iteration 615
Iteration 616
Iteration 617
Iteration 618
Iteration 619
Iteration 620
Iteration 621
Iteration 622
Iteration 623
Iteration 624
Iteration 625
Iteration 626
Iteration 627
Iteration 628
Iteration 629
Iteration 630
Iteration 631
Iteration 632
Iteration 633
Iteration 634
Iteration 635
Iteration 636
Iteration 637
Iteration 638
Iteration 639
Iteration 640
Iteration 641
Iteration 642
Iteration 643
Iteration 644
Iteration 645
Iteration 646
Iteration 647
Iteration 648
Iteration 649
Iteration 650
Iteration 651
Iteration 652
Iteration 653
Iteration 654
Iteration 655
Iteration 656
Iteration 657
Iteration 658
Iteration 659
Iteration 660
Iteration 661
Iteration 662
Iteration 663
Iterat

In [126]:
# NLN Training on use case -- ver2

## select two variables > select logic gate > get output and repeat until result found > if result not match, continue
## if result  match, try the same logic gate sructure with all the other rows, store accuracy score (how many match)

# parse variable by variable
## Question: should we only go through single AGe var, or all of them? 
## -- probably all of them, or else how distinguish one 1 from another?
## -- does not being of a certain age add information bits?
## -- age is already given and can be used to auto-filter; why unfilter?

from random import randint
import itertools
from tqdm import tnrange, tqdm_notebook, tqdm
from time import sleep
# for index, row in encodedData.iterrows():
# for i in tnrange(3, desc='1st loop'):
#     for j in tqdm_notebook(range(100), desc='2nd loop'):
#         sleep(0.01)

logicName = ['AND', 'OR', 'NAND', 'XOR']
logicNN = [nnAND, nnOR, nnNAND, nnXOR] # nnNOT does not work at the moment
# logicName = ['AND', 'OR', 'XOR']
# logicNN = [nnAND, nnOR, nnXOR] # nnNOT does not work at the moment

logicHistory = []
varHistory = []
logicnameHistory = []

varLogs = []
logicLogs = []
logicnameLogs = []
accuracyLogs = []
accuracyInverseLogs = []

varName = list(encodedData.columns)
placeholderList = []

# select variables
for i in tnrange(100): # n iterations
    
    print("Iteration " + str(i+1))
        
    logicHistory = []
    logicnameHistory = []
    varHistory = []
    
    # new logic
    # set random number of logic gates to render
    # randomly select a gate
        # randomly select 2 variables to get output
        # apply gate and get output
        # randomly select another variable
        # use output and variabel to get output
    # continue to get output until reach max logic count
    # cancel feature: compare value to see if both match output
    # see if this logic machine can apply to other rows
        # set counter to increase if matching output
        
    maxGatesCount = 50
    randomGatesCount = randint(0, maxGatesCount)
    gateCounter = 0
    firstIteration = True
    
    # logic machine creation
    
    while gateCounter < randomGatesCount:
        
#         print("Iteration " + str(i+1))
        
#         logicHistory = []
#         logicnameHistory = []
#         varHistory = []
        
        if firstIteration == True:
            
            randomVarindex = randint(0, len(varName)-1)
            var = varName[randomVarindex]
            
            placeholderList = [item for item in varName if item!=var]
            
            randomVarindex = randint(0, len(placeholderList)-1)
            var2 = placeholderList[randomVarindex]
    
            varHistory.append(var); varHistory.append(var2)
        
            randomIndex = randint(0, len(logicName)-1)
            gate = logicNN[randomIndex]
        
            input1 = encodedData[var][0]
            input2 = encodedData[var2][0]
            
            output = gate.query([input1, input2])[0][0]
            
            logicHistory.append(gate); logicnameHistory.append(logicName[randomIndex])
            
            firstIteration = False
            gateCounter += 1
            
        else:
            
            randomVarindex = randint(0, len(varName)-1)
            var = varName[randomVarindex]

            varHistory.append(var)
            
            randomIndex = randint(0, len(logicName)-1)
            gate = logicNN[randomIndex]
        
            input1 = encodedData[var][0]
            
            output = gate.query([output, input1])[0][0]
            
            logicHistory.append(gate); logicnameHistory.append(logicName[randomIndex])
            
            gateCounter += 1
            
            
        # logic machine backtesting
        
        for index, row in encodedDataComplete.iterrows():
            correctCount = 0
            correctInverseCount = 0
            intCounter = 0
            v1 = row[varHistory[intCounter]]
            while intCounter < len(varHistory)-1:

                lg = logicHistory[intCounter]
                intCounter +=1
                v2 = row[varHistory[intCounter]]

                otpt = lg.query([v1, v2])[0][0]
                v1 = otpt

            if round(otpt) == row[list(encodedDataComplete.columns)[-1]]:
                correctCount+=1
            if round(otpt) != row[list(encodedDataComplete.columns)[-1]]:
                correctInverseCount+=1

        # store accuracy score, store history logs
        accuracy = correctCount / float(len(encodedDataComplete))
        accuracyInverse = correctInverseCount / float(len(encodedDataComplete))
        
        accuracyLogs.append(accuracy)
        accuracyInverseLogs.append(accuracyInverse)
        varLogs.append(varHistory)
        logicLogs.append(logicHistory)
        logicnameLogs.append(logicnameHistory)

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration 77
Iteratio

In [127]:
logsDF = pd.DataFrame()
logsDF['varStructure'] = varLogs
logsDF['logicStructure'] = logicnameLogs
logsDF['logicMachine'] = logicLogs
logsDF['accuracyTheoreticalInverse'] = [1.0 - x for x in accuracyLogs]
# logsDF['accuracy'] = accuracyLogs
# logsDF['accuracyInverse'] = accuracyInverseLogs
logsDF = logsDF.sort_values(by=['accuracyTheoreticalInverse'], ascending=True)
logsDF

,varStructure,logicStructure,logicMachine,accuracyTheoreticalInverse
1285,"[Age, Married, Gender, Age, Gender, Married, M...","[XOR, XOR, XOR, OR, NAND, AND, OR, AND, NAND, ...",[<__main__.neuralNetwork object at 0x000001DEF...,0.962963
1477,"[Married, Gender, Age, Married, Age, Age, Marr...","[OR, XOR, OR, OR, XOR, NAND, XOR, NAND, AND, A...",[<__main__.neuralNetwork object at 0x000001DEF...,0.962963
1476,"[Married, Gender, Age, Married, Age, Age, Marr...","[OR, XOR, OR, OR, XOR, NAND, XOR, NAND, AND, A...",[<__main__.neuralNetwork object at 0x000001DEF...,0.962963
1475,"[Married, Gender, Age, Married, Age, Age, Marr...","[OR, XOR, OR, OR, XOR, NAND, XOR, NAND, AND, A...",[<__main__.neuralNetwork object at 0x000001DEF...,0.962963
1472,"[Married, Gender, Age, Married, Age, Age, Marr...","[OR, XOR, OR, OR, XOR, NAND, XOR, NAND, AND, A...",[<__main__.neuralNetwork object at 0x000001DEF...,0.962963
1471,"[Married, Gender, Age, Married, Age, Age, Marr...","[OR, XOR, OR, OR, XOR, NAND, XOR, NAND, AND, A...",[<__main__.neuralNetwork object at 0x000001DEF...,0.962963
1470,"[Married, Gender, Age, Married, Age, Age, Marr...","[OR, XOR, OR, OR, XOR, NAND, XOR, NAND, AND, A...",[<__main__.neuralNetwork object at 0x000001DEF...,0.962963
1469,"[Married, Gender, Age, Married, Age, Age, Marr...","[OR, XOR, OR, OR, XOR, NAND, XOR, NAND, AND, A...",[<__main__.neuralNetwork object at 0x000001DEF...,0.962963
1467,"[Married, Gender, Age, Married, Age, Age, Marr...","[OR, XOR, OR, OR, XOR, NAND, XOR, NAND, AND, A...",[<__main__.neuralNetwork object at 0x000001DEF...,0.962963
1464,"[Married, Gender, Age, Married, Age, Age, Marr...","[OR, XOR, OR, OR, XOR, NAND, XOR, NAND, AND, A...",[<__main__.neuralNetwork object at 0x000001DEF...,0.962963


In [129]:
logsDF.iloc[0,:]

varStructure                  [Age, Married, Gender, Age, Gender, Married, M...
logicStructure                [XOR, XOR, XOR, OR, NAND, AND, OR, AND, NAND, ...
logicMachine                  [<__main__.neuralNetwork object at 0x000001DEF...
accuracyTheoreticalInverse                                             0.962963
Name: 1285, dtype: object

In [130]:
# logic gate visualization
logsDF2 = logsDF.reset_index(drop=True)
selectedmachine = logsDF2.iloc[0,:]
first = True
last = False
index = 0
while last == False:
    if first == True:
        first = False
        print(selectedmachine['varStructure'][index], " ", selectedmachine['logicStructure'][index], " ", selectedmachine['varStructure'][index+1])
        index+=1
    else:
        print("Output ", selectedmachine['logicStructure'][index], " ", selectedmachine['varStructure'][index])
        index+=1
        if index == len(selectedmachine['varStructure'])-1:
            last=True

Age   XOR   Married
Output  XOR   Married
Output  XOR   Gender
Output  OR   Age
Output  NAND   Gender
Output  AND   Married
Output  OR   Married
Output  AND   Gender
Output  NAND   Married
Output  NAND   Gender
Output  XOR   Age
Output  OR   Gender
Output  NAND   Gender
Output  NAND   Gender
Output  NAND   Gender
Output  XOR   Gender
Output  NAND   Gender
Output  XOR   Gender
Output  AND   Age
Output  NAND   Age
Output  XOR   Married
Output  AND   Gender
Output  XOR   Age
Output  NAND   Married
Output  AND   Gender
Output  AND   Gender
Output  NAND   Age
Output  AND   Married
Output  AND   Age
Output  OR   Age
Output  NAND   Gender
Output  NAND   Gender
Output  AND   Age
Output  OR   Age
Output  NAND   Gender
Output  NAND   Married
Output  XOR   Married
Output  XOR   Gender
Output  XOR   Married
Output  NAND   Gender
Output  NAND   Married
Output  NAND   Gender
Output  OR   Married
Output  XOR   Gender


In [132]:
accuracy = logsDF.iloc[0,:][-1]
print("Neural Logic Network accuracy ", accuracy)

Neural Logic Network accuracy  0.962962962962963


In [18]:
logsDF.to_csv("logs_NLN.csv")